# Representations

In [82]:
import os
import re
import glob
import pickle
import logging
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from datetime import datetime
from collections import Counter
import pandas as pd
import ftfy
import spacy
from textacy.datasets.wikipedia import Wikipedia, strip_markup
from gensim.models import Word2Vec, FastText
from gensim.models.word2vec import PathLineSentences
from tqdm import tqdm_notebook as tqdm
tqdm.monitor_interval = 0

logging.basicConfig(filename='log.log', filemode='w', level=logging.INFO, 
                        format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p')
logging.getLogger("gensim").setLevel(logging.WARNING)

WIKI_DATA_DIR = '/home/bacon/miniconda3/lib/python3.6/site-packages/textacy/data/wikipedia'

## Download and prepare Wikipedia data

In [72]:
class WikipediaCorpus:
    
    MIN_NUM_SENTS_PER_ARTICLE = 10
    MIN_NUM_WORDS_IN_SENT = 5
    MAX_NUM_WORDS_IN_SENT = 70
    
    SENT_ENDS = ['.', '!', '?']
    TABLE_PREFIX = re.compile(r'\s*(\{\))|(\|)|(\|\})')
    TAG = re.compile('<[^<>]+>')
    
    BAD_POS = ['INTJ', 'SYM', 'X']
    
    def __init__(self, lg):
        self.lg = lg
        nlp = spacy.load(lg, disable=['tagger', 'parser', 'ner', 'textcat'])
        self.tokenizer = nlp.tokenizer
        self.out_dir = os.path.join(lg, 'wikipedia')
        os.makedirs(self.out_dir) # errors if out_dir already exists so I don't re-write data
    
    def download(self):
        wp = Wikipedia(lang=self.lg, version='latest', data_dir=WIKI_DATA_DIR)
        _ = wp.download()
        return
    
    def prepare(self, N):
        self.N = N
        self.num_sents = 0
        self.num_tokens = 0
        self.freq_dist = Counter()
        wp = Wikipedia(lang=self.lg, version='latest', data_dir=WIKI_DATA_DIR)
        for i, _, content in tqdm(wp):
            n, text = self.prepare_one_article(content)
            if text:
                self.num_sents += n
                tokens = text.split()
                self.num_tokens += len(tokens)
                self.freq_dist.update(tokens)
                self.write(text, i)
                if self.num_sents >= self.N:
                    self.write_metadata()
                    return  
    
    def prepare_one_article(self, content):
        content = '\n'.join([line for line in content.split('\n') if not self.TABLE_PREFIX.match(line)])
        content = strip_markup(content)
        lines = self.clean_lines(content)
        sentences = []
        for line in lines:
            for sent in self.tokenize(line):
                sentences.append(sent)
        article_length = len(sentences)
        if article_length >= self.MIN_NUM_SENTS_PER_ARTICLE:
            return article_length, '\n'.join(sentences).lower()
        return 0, ''
    
    def clean_lines(self, content):
        content = ftfy.fix_text(content)
        for line in content.split('\n'):
            line = self.TAG.sub('', line.strip())
            if line and line[0].isalnum():
                char_ratio = float(sum(ch.islower() for ch in line)) / sum(not ch.isspace() for ch in line)
                if char_ratio > 0.9:
                    yield line
    
    def tokenize(self, content):
        tokens = []
        for token in self.tokenizer(content):
            token = token.text
            tokens.append(token)
            if token in self.SENT_ENDS:
                if self.MIN_NUM_WORDS_IN_SENT <= len(tokens) <= self.MAX_NUM_WORDS_IN_SENT:
                    yield ' '.join(tokens)
                tokens = []
        if tokens:
            if self.MIN_NUM_WORDS_IN_SENT <= len(tokens) <= self.MAX_NUM_WORDS_IN_SENT:
                yield ' '.join(tokens)
                
#     def preprocess(self, doc):
#         result = []
#         for t in doc:
#             is_bad_pos = t.pos_ in self.BAD_POS
#             is_bad_punct = t.is_punct and (len(t.text) > 1)
#             is_bad_token = t.like_url or t.like_email or t.is_space or is_bad_pos or is_bad_punct
#             is_num = t.is_digit or t.like_num
#             if is_bad_token:
#                 break
#             elif is_num:
#                 text = 'num'
#             else:
#                 text = t.text.lower()
#             result.append(text)
#         return result
    
    def write(self, text, i):
        fname = os.path.join(self.out_dir, '{}.txt'.format(i))
        with open(fname, encoding='utf-8', mode='w') as f:
            f.write(text.replace('\xa0',' '))
    
    def write_metadata(self):
        metadata = {'num_sents': self.num_sents, 'num_tokens': self.num_tokens,
                    'freq_dist': self.freq_dist, 'N': self.N, 'max_length': self.MAX_NUM_WORDS_IN_SENT}
        fname = os.path.join(self.lg, 'metadata.pkl')
        with open(fname, mode='wb') as f:
            pickle.dump(metadata, f)

In [73]:
def clear_dirs(lgs):
    for lg in lgs:
        cmd = 'rm -rf {}/wikipedia'.format(lg)
        os.system(cmd)
        cmd = 'rm {}/metadata.pkl'.format(lg)
        os.system(cmd)

def main(lgs, N):
    clear_dirs(lgs)
    for lg in lgs:
        start = datetime.now()
        corpus = WikipediaCorpus(lg)
        #corpus.download()
        corpus.prepare(N)
        end = datetime.now()
        msg = 'Processing {} sentences for {} took {}'.format(N, lg, end-start)
        logging.info(msg)

In [74]:
main(['en'], 1000)


0it [00:00, ?it/s]
2it [00:00,  4.01it/s]
12it [00:00, 15.66it/s]
39it [00:00, 44.99it/s]
